### Modelling Notebook

In [1]:
# importing external libraries
from pathlib import Path
import os
import pandas as pd
import pickle
import json
import matplotlib.pyplot as plt
import numpy as np
import logging
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn import set_config
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier
set_config(transform_output = "pandas")

# Importing function to load data

# Making sure any changes are instantly added
%load_ext autoreload
%autoreload 2

from Modules.load_data import load_data
from Modules.preprocessing import missing_summary, merge_dfs, dollar_to_int, find_unique_values
from Modules.plotting import Plotter
from Modules.transforming import *
from Modules.modelling import MLearner

# Importing Pipelines
from Modules.Pipelines import Pipeline1, Pipeline2, Pipeline3


/home/as3620/FODS_coursework/LoanPrediction/data
/home/as3620/FODS_coursework/LoanPrediction


In [2]:
# Obtaining Root dir

root = str(Path.cwd())

# Obtaining seed from config.yaml

# Load the config file
with open(root + "/config.yaml", "r") as file:
    config = yaml.safe_load(file)

seed = config["global"]["seed"]

#print(f"seed: {seed}")

# Set global seeds for reproducibility
random.seed(seed)        
np.random.seed(seed)     

# Use the seed in scikit-learn
random_state = check_random_state(seed)

In [3]:
# Obtaining absolute path to data folder

data_folder = str(Path(os.getcwd()) / "data")

In [4]:
# Loading the data from pickle

merged_df = pd.read_pickle(data_folder + "/merged_data.pkl")

In [6]:
# Defining params for CV grid search

params_simple = {"penalty": [None], "solver":["saga"], "class_weight": ["balanced"], "max_iter":[1000]}

params_enhanced = [
    {"penalty": [None], "solver":["saga"], "class_weight": [None, "balanced"]},
    {"penalty": ["elasticnet"], "l1_ratio" : np.linspace(0,1,10).tolist(), 
     "C": np.linspace(0.01,1,10).tolist(), "solver":["saga"], "class_weight": ["balanced"], "max_iter":[1000]}    
]


We will first use the simple parametrs to save computational time and answer the following questions:
1. Is Pipeline2 better than Pipeline1. Is better granularity for date-like columns more predictive?
2. Does using a dataset with a larger amount of "Non-fraudulent transactions" improve performance? As dataset is very imbalanced.

Then we will carry out hyperparameter tuning on GLM and LGBM estimators

### Pipeline1

Transforming date into a time series

In [22]:
ML_pipe1 = MLearner(dataset=merged_df, transformation_pipeline=Pipeline1, params=params_simple, estimator=LogisticRegression(random_state=seed))
ML_pipe1.fit()
ML_pipe1.predict()

% of fraudulent transactions in y_train: 0.1302501302761855
% of fraudulent transactions in y_test: 0.1302954509926528

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5] END class_weight=balanced, max_iter=1000, penalty=None, solver=saga;, score=0.897 total time=   0.6s
[CV 2/5] END class_weight=balanced, max_iter=1000, penalty=None, solver=saga;, score=0.888 total time=   0.6s
[CV 3/5] END class_weight=balanced, max_iter=1000, penalty=None, solver=saga;, score=0.901 total time=   0.6s
[CV 4/5] END class_weight=balanced, max_iter=1000, penalty=None, solver=saga;, score=0.894 total time=   0.6s
[CV 5/5] END class_weight=balanced, max_iter=1000, penalty=None, solver=saga;, score=0.895 total time=   0.6s
Best parameters found: {'class_weight': 'balanced', 'max_iter': 1000, 'penalty': None, 'solver': 'saga'}
score on training set: 0.8952503091650255
score on testing set: 0.8457047539616347


### Pipeline2

Decomposing date-like features into hours, weeks, etc.

In [24]:
ML_pipe2 = MLearner(dataset=merged_df, transformation_pipeline=Pipeline2, params=params_simple, estimator=LogisticRegression(random_state=seed))
ML_pipe2.fit()
ML_pipe2.predict()

% of fraudulent transactions in y_train: 0.1302501302761855
% of fraudulent transactions in y_test: 0.1302954509926528

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5] END class_weight=balanced, max_iter=1000, penalty=None, solver=saga;, score=0.905 total time=   0.7s
[CV 2/5] END class_weight=balanced, max_iter=1000, penalty=None, solver=saga;, score=0.904 total time=   0.7s
[CV 3/5] END class_weight=balanced, max_iter=1000, penalty=None, solver=saga;, score=0.915 total time=   0.7s
[CV 4/5] END class_weight=balanced, max_iter=1000, penalty=None, solver=saga;, score=0.905 total time=   0.7s
[CV 5/5] END class_weight=balanced, max_iter=1000, penalty=None, solver=saga;, score=0.914 total time=   0.7s
Best parameters found: {'class_weight': 'balanced', 'max_iter': 1000, 'penalty': None, 'solver': 'saga'}
score on training set: 0.909183531700088
score on testing set: 0.8632022471910112


### Pipeline 3

As we can see the date transformations improve performance. We will now see what an effect increasing the size of the dataset has.

In [7]:
# Set up logging configuration

reduction_p = [0.00125, 0.0025, 0.005, 0.01, 0.05]

for p in reduction_p:
    print(f"\nMachine learning with {p} of dataset...\n")
    merged_df3_copy = Target0_Reducer(percentage=p).fit_transform(merged_df)

    ML_pipe3 = MLearner(dataset=merged_df3_copy, transformation_pipeline=Pipeline3, params=params_simple, estimator=LogisticRegression(random_state=seed), scoring="f1")
    ML_pipe3.fit()
    ML_pipe3.predict()
    
    


Machine learning with 0.00125 of dataset...

% of fraudulent transactions in y_train: 0.5446467509812473
% of fraudulent transactions in y_test: 0.5463614063777597

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5] END class_weight=balanced, max_iter=1000, penalty=None, solver=saga;, score=0.989 total time=   1.2s
[CV 2/5] END class_weight=balanced, max_iter=1000, penalty=None, solver=saga;, score=0.990 total time=   1.4s
[CV 3/5] END class_weight=balanced, max_iter=1000, penalty=None, solver=saga;, score=0.988 total time=   1.3s
[CV 4/5] END class_weight=balanced, max_iter=1000, penalty=None, solver=saga;, score=0.985 total time=   1.4s
[CV 5/5] END class_weight=balanced, max_iter=1000, penalty=None, solver=saga;, score=0.986 total time=   1.4s
Best parameters found: {'class_weight': 'balanced', 'max_iter': 1000, 'penalty': None, 'solver': 'saga'}
score on training set: 0.9878182725911133
score on testing set: 0.9453357100415924

Machine learning with 0.0025 of data

/home/as3620/miniconda3/envs/FODS_coursework/lib/python3.13/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV 1/5] END class_weight=balanced, max_iter=1000, penalty=None, solver=saga;, score=0.560 total time= 1.2min


/home/as3620/miniconda3/envs/FODS_coursework/lib/python3.13/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV 2/5] END class_weight=balanced, max_iter=1000, penalty=None, solver=saga;, score=0.564 total time= 1.2min


...

/home/as3620/miniconda3/envs/FODS_coursework/lib/python3.13/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV 4/5] END class_weight=balanced, max_iter=1000, penalty=None, solver=saga;, score=0.610 total time= 1.1min


/home/as3620/miniconda3/envs/FODS_coursework/lib/python3.13/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV 5/5] END class_weight=balanced, max_iter=1000, penalty=None, solver=saga;, score=0.604 total time= 1.2min
Best parameters found: {'class_weight': 'balanced', 'max_iter': 1000, 'penalty': None, 'solver': 'saga'}
score on training set: 0.5567271907071163
score on testing set: 0.5220726095108232


/home/as3620/miniconda3/envs/FODS_coursework/lib/python3.13/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Interestingly we see that as we increase the dataframe size performance decreases. Perhaps the model cannot generalise well to so few fraudulent transactions. In addition Logit fails to converge. We propose to use a balanced dataset.

In [11]:
# Obtaining score for balanced class

# Set up logging configuration
logging.basicConfig(level=logging.CRITICAL)


merged_df3_copy = Target0_Reducer(balanced=True).fit_transform(merged_df)

ML_pipe3 = MLearner(dataset=merged_df3_copy, transformation_pipeline=Pipeline3, params=params_simple, estimator=LogisticRegression(), scoring="f1")
ML_pipe3.fit()
ML_pipe3.predict()

DEBUG:root:X_train indices: Index([19973, 18804, 26056, 23547, 23256, 11170, 20521,  3638,  6547,  7827,
       ...
       16850,  6265, 22118, 11284, 11964, 21575,  5390,   860, 15795, 23654],
      dtype='int64', length=19998)
DEBUG:root:X_test indices: Index([20966, 26365, 20166,  8763,  7335, 25135, 18791, 12228, 11812, 19142,
       ...
        8390, 17202,  6161,  7801, 26575,  3388,  3310, 23973, 18548, 24660],
      dtype='int64', length=6666)


% of fraudulent transactions in y_train: 0.5011501150115012
% of fraudulent transactions in y_test: 0.4965496549654965

Best parameters found: {'class_weight': 'balanced', 'max_iter': 1000, 'penalty': None, 'solver': 'saga'}
score on training set: 0.9849840891010342
score on testing set: 0.9477623110312827


In [13]:
# Now hyperparameter tuning

# Set up logging configuration
logging.basicConfig(level=logging.CRITICAL)


merged_df3_copy = Target0_Reducer(balanced=True).fit_transform(merged_df)

ML_pipe3 = MLearner(dataset=merged_df3_copy, transformation_pipeline=Pipeline3, params=params, estimator=LogisticRegression(), scoring="accuracy")
ML_pipe3.fit()
ML_pipe3.predict()

DEBUG:root:X_train indices: Index([18137, 19765, 18101, 26603,  3625, 16291,  5454, 12365, 20777, 15780,
       ...
        5566,   284, 23527,  5093, 14301, 15058, 25230, 12134,  2082,  7590],
      dtype='int64', length=19998)
DEBUG:root:X_test indices: Index([18988, 24283,  4969, 12047, 23953,   812, 23493, 10565, 20631, 22202,
       ...
        5582, 12400,  4382, 16181, 26619,  6945,  2818,  5969, 21674, 21480],
      dtype='int64', length=6666)


% of fraudulent transactions in y_train: 0.502050205020502
% of fraudulent transactions in y_test: 0.49384938493849384



/home/as3620/miniconda3/envs/FODS_coursework/lib/python3.13/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/as3620/miniconda3/envs/FODS_coursework/lib/python3.13/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/as3620/miniconda3/envs/FODS_coursework/lib/python3.13/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/as3620/miniconda3/envs/FODS_coursework/lib/python3.13/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/as3620/miniconda3/envs/FODS_coursework/lib/python3.13/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means

Best parameters found: {'class_weight': None, 'penalty': None, 'solver': 'saga'}
score on training set: 0.9857485748574858
score on testing set: 0.9426942694269427


/home/as3620/miniconda3/envs/FODS_coursework/lib/python3.13/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


### LGBM set up with Pipeline3



In [ ]:
# LGBM params

lgbm_params =     {"learning_rate": [0.01, 0.02],
                   "n_estimators": [50],
                   #"reg_alpha": np.linspace(0,1,10).tolist(),
                   #"reg_lambda": np.linspace(0,1,10).tolist(),
                   }


merged_lgbm = Target0_Reducer(balanced=True).fit_transform(merged_df)

ML_pipe3 = MLearner(dataset=merged_lgbm, transformation_pipeline=Pipeline3, params=lgbm_params, estimator=LGBMClassifier(), scoring="accuracy",cv=3)
ML_pipe3.fit()
ML_pipe3.predict()

% of fraudulent transactions in y_train: 0.5025502550255025
% of fraudulent transactions in y_test: 0.49234923492349236

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 6700, number of negative: 6632
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.030179 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1703
[LightGBM] [Info] Number of data points in the train set: 13332, number of used features: 23
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.502550 -> initscore=0.010201
[LightGBM] [Info] Start training from score 0.010201
[CV 1/3] END learning_rate=0.01, n_estimators=50;, score=0.988 total time=  45.4s
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 6700, number of negative: 6632
[LightGBM] [Info] A

: 

In [ ]:
lgbm_params =     {"learning_rate": [0.02, 0.1],
                   "n_estimators": [50],
                   #"reg_alpha": np.linspace(0,1,10).tolist(),
                   #"reg_lambda": np.linspace(0,1,10).tolist(),
                   }


merged_lgbm = Target0_Reducer(balanced=True).fit_transform(merged_df)

ML_pipe3 = MLearner(dataset=merged_lgbm, transformation_pipeline=Pipeline3, params=lgbm_params, estimator=LGBMClassifier(), scoring="accuracy",cv=3)
ML_pipe3.fit()
ML_pipe3.predict()

% of fraudulent transactions in y_train: 0.5011501150115012
% of fraudulent transactions in y_test: 0.4965496549654965

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 6682, number of negative: 6650
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.029896 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1718
[LightGBM] [Info] Number of data points in the train set: 13332, number of used features: 23
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.501200 -> initscore=0.004800
[LightGBM] [Info] Start training from score 0.004800
[CV 1/3] END learning_rate=0.02, n_estimators=50;, score=0.987 total time=  43.3s
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 6681, number of negative: 6651
[LightGBM] [Info] Au

: 

In [5]:
lgbm_params =     {"learning_rate": [0.1, 0.2],
                   "n_estimators": [50],
                   #"reg_alpha": np.linspace(0,1,10).tolist(),
                   #"reg_lambda": np.linspace(0,1,10).tolist(),
                   }


merged_lgbm = Target0_Reducer(balanced=True).fit_transform(merged_df)

ML_pipe3 = MLearner(dataset=merged_lgbm, transformation_pipeline=Pipeline3, params=lgbm_params, estimator=LGBMClassifier(), scoring="accuracy",cv=3)
ML_pipe3.fit()
ML_pipe3.predict()

% of fraudulent transactions in y_train: 0.5011501150115012
% of fraudulent transactions in y_test: 0.4965496549654965

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 6682, number of negative: 6650
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.027913 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1718
[LightGBM] [Info] Number of data points in the train set: 13332, number of used features: 23
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.501200 -> initscore=0.004800
[LightGBM] [Info] Start training from score 0.004800
[CV 1/3] END learning_rate=0.1, n_estimators=50;, score=0.989 total time=  47.3s
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 6681, number of negative: 6651
[LightGBM] [Info] Aut

In [1]:
lgbm_params =     {"learning_rate": [0.2],
                   "n_estimators": [100],
                   #"reg_alpha": np.linspace(0,1,10).tolist(),
                   #"reg_lambda": np.linspace(0,1,10).tolist(),
                   }


merged_lgbm = Target0_Reducer(balanced=True).fit_transform(merged_df)

ML_pipe3 = MLearner(dataset=merged_lgbm, transformation_pipeline=Pipeline3, params=lgbm_params, estimator=LGBMClassifier(), scoring="accuracy",cv=3)
ML_pipe3.fit()
ML_pipe3.predict()

NameError: name 'Target0_Reducer' is not defined